<a href="https://colab.research.google.com/github/lbdlab/NGS-LBD/blob/master/notebooks/HPO_Ontology_QUERY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neo4j pandas seaborn matplotlib


     |████████████████████████████████| 89 kB 2.7 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.0-py3-none-any.whl size=114860 sha256=b5c2729c90483ddbff6f9a0d5d3101da0343719064c008f6618892b233ebc944
  Stored in directory: /root/.cache/pip/wheels/b8/cb/e4/d34e10b40d33911c281a6e9aa038d54ca7435310529b3f6042
Successfully built neo4j


In [2]:
from neo4j import GraphDatabase
import pandas as pd

In [3]:
# Connect to Neo4j
driver = GraphDatabase.driver('neo4j://212.235.239.171:22333', auth=('neo4j', 'ngslbd2020'))

def run_query(query, params={}):
    with driver.session(database="hpo") as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [7]:
run_query("""
MATCH p=(h:HPO)-[:IS_A*]->(parent)
WHERE h.name = "Abnormal prostate morphology"
RETURN p, h.id, h.name, parent.id, parent.name
""")

,p,h.id,h.name,parent.id,parent.name
0,(()),HP:0008775,Abnormal prostate morphology,HP:0000022,Abnormality of male internal genitalia
1,"((), ())",HP:0008775,Abnormal prostate morphology,HP:0010461,Abnormality of the male genitalia
2,"((), (), ())",HP:0008775,Abnormal prostate morphology,HP:0012243,Abnormal reproductive system morphology
3,"((), (), (), ())",HP:0008775,Abnormal prostate morphology,HP:0000078,Abnormality of the genital system
4,"((), (), (), (), ())",HP:0008775,Abnormal prostate morphology,HP:0000119,Abnormality of the genitourinary system
5,"((), (), (), (), (), ())",HP:0008775,Abnormal prostate morphology,HP:0000118,Phenotypic abnormality
6,"((), (), (), (), (), (), ())",HP:0008775,Abnormal prostate morphology,HP:0000001,All
7,"((), ())",HP:0008775,Abnormal prostate morphology,HP:0000812,Abnormal internal genitalia
8,"((), (), ())",HP:0008775,Abnormal prostate morphology,HP:0012243,Abnormal reproductive system morphology
9,"((), (), (), ())",HP:0008775,Abnormal prostate morphology,HP:0000078,Abnormality of the genital system


In [8]:
run_query("""
MATCH p=(h:HPO)-[:IS_A*0..]->(parents)
WHERE h.name = "Abnormal prostate morphology"
RETURN collect(distinct parents.name) AS result
""")['result'].to_list()

[['Abnormal prostate morphology',
  'Abnormality of male internal genitalia',
  'Abnormality of the male genitalia',
  'Abnormal reproductive system morphology',
  'Abnormality of the genital system',
  'Abnormality of the genitourinary system',
  'Phenotypic abnormality',
  'All',
  'Abnormal internal genitalia']]